# Gauss Seidel Method

Gauss Seidel method is another stationary iterative method for solving a linear system $$A{\bf x}={\bf b}$$ It splits the matrix $A=D+U+L$, where $D$ represents the diagonal of the matrix $A$, and $U$ and $L$ represent the upper and lower triangular parts of the matrix $A$, respectively. Let ${\bf x}^{(k)}$ denotes the $k$-th iteration of the approximated solution, then the Gauss Seidel method reads $${\bf x}^{(k+1)}=(L+D)^{-1}\left({\bf b}-U{\bf x}^{(k)}\right)$$

Or componentwisely, it can be written as

$$x_i^{(k+1)}=\frac{1}{a_{ii}}\left(b_i-\sum_{j=0}^{i-1}a_{ij}x_j^{(k+1)}-\sum_{j=i+1}^{n}a_{ij}x_j^{(k)}\right)$$

* Now, let's try to use Gauss Seidel Method to solve a one dimentional Poissons equation:
$$-u_{xx}=\pi^2\sin{\pi x} \quad\text{ for } x\in(0,1)$$
$$\text{with}\quad u(0)=u(1)=0$$


* Let's try to use Gauss Seidel Method to solve a $2D$ Poissons equation. 
$$-u_{xx}-u_{yy}=2\pi^2\sin{\pi x}\cos{\pi y} \quad\text{ for } x\in(0,1)\text{ and } y\in(0,1)$$
$$\text{with}\quad u(0,y)=u(1,y)=0$$
$$\text{and }\quad u(x,0)=\sin{\pi x}\quad u(x,1)=-\sin{\pi x}$$

### Error Analysis on Jacobi and Gauss Seidel methods

For a linear system $A{\bf x}={\bf b}$
* the error is defined as: $$e^{(k)}=\|{\bf x}^{(k)}-{\bf x}\|_{2}=\sqrt{\sum_{i=1}^{n}\left(x^{(k)}_i-x_i\right)^2}$$
* the residue is defined as: $$r^{(k)}=\|{\bf b}-A{\bf x}^{(k)}\|_{2}=\sqrt{\sum_{i=1}^{n}\left(b_i-\sum_{j=1}^{n}A_{ij}x_j^{(k)}\right)^2}$$

* Now instead of using for-loops, we can use while-loops to implement Jacobi and Gauss Seidel methods. 
* The stopping criteria for the while loop should be $r^{(k)}<\epsilon$